## Deep Learning Model - CNN Multi-Label Text Classification
### BERT with Sub Themes (Right Now Ran for Main Themes)

In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D, Input
from keras.layers.recurrent import GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from tensorflow.keras import layers
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Using plaidml.keras.backend backend.


### Loading Data

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

### Creating a Unified Dataframe for CNN Ready Model

In [3]:
df = pd.concat([X_train_Q1, y_train_Q1.iloc[:,0:12]], axis = 1)

In [4]:
df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
data_df = df

In [6]:
data_df.shape

(10376, 13)

### Pre-processing

#### General

In [7]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [8]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [9]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in data_df['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
data_df['preprocessed_comments']=preprocessed_synopsis

In [10]:
data_df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH,preprocessed_comments
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0,real diversity you need create seats table mea...
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0,keep building warmer provide warm water bathroom
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0,better communication top
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0,would beneficial management not micro manage
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0,education applicable job


### Splitting into Train and Test

In [11]:
X_train = data_df[['preprocessed_comments']]
y_train = data_df.drop(['Comment', 'preprocessed_comments'], axis=1)

### Preparing Labels

**Max length of sentence**

In [12]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(data_df['Comment'].apply(max_len))
max_len

150

### Vocab Size

In [13]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

11919


**BERT Encoder**

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [8]:
bert_encoder = SentenceTransformer('bert-large-nli-mean-tokens')

In [9]:
sentence_embeddings = bert_encoder.encode(X_train_Q1['Comment'])

In [10]:
embedding_matrix = np.asarray(sentence_embeddings)

In [11]:
embedding_matrix.shape

(10376, 1024)

In [12]:
# saving embeddings to file
np.save('../models/bert_x_train_embeddings', embedding_matrix)

## Modelling CNN

#### Defining Model

## BERT CNN

In [ ]:
y_train = np.array(y_train)

max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 1
embed_size = 512 # for universal sentence encoder
n_class = 12

In [ ]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[embedding_matrix],
                        trainable=False, input_length=maxlen))




model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

In [ ]:
padded_docs_train.shape

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

# Train Model
model.fit(padded_docs_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.05)

### Testing Simple Keras Model

In [19]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 150
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10
embed_size = 1024 # for BERT Large
n_class = 12

In [20]:
# Build the model
model = Sequential()
model.add(Dense(max_features, input_shape=(embed_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(n_class))
model.add(Activation('sigmoid'))

INFO:plaidml:Opening device "metal_amd_radeon_pro_555x.0"
INFO:plaidml:Opening device "metal_amd_radeon_pro_555x.0"


In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(embedding_matrix, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.15)

Train on 8819 samples, validate on 1557 samples
Epoch 1/10
8250/8819 [===========================>..] - ETA: 0s - loss: 0.2863 - acc: 0.8998

/Users/karan/anaconda3/lib/python3.7/site-packages/numpy/ctypeslib.py:436: RuntimeWarning: Invalid PEP 3118 format string: '&<f'
  return array(obj, copy=False)


8819/8819 [==============================] - 4s 416us/step - loss: 0.2812 - acc: 0.9011 - val_loss: 0.2201 - val_acc: 0.9170
Epoch 2/10
8819/8819 [==============================] - 2s 255us/step - loss: 0.2017 - acc: 0.9230 - val_loss: 0.2101 - val_acc: 0.9205
Epoch 3/10
8819/8819 [==============================] - 2s 253us/step - loss: 0.1892 - acc: 0.9276 - val_loss: 0.2022 - val_acc: 0.9239
Epoch 4/10
8819/8819 [==============================] - 2s 249us/step - loss: 0.1783 - acc: 0.9313 - val_loss: 0.1991 - val_acc: 0.9242
Epoch 5/10
8819/8819 [==============================] - 2s 254us/step - loss: 0.1700 - acc: 0.9338 - val_loss: 0.2022 - val_acc: 0.9241
Epoch 6/10
8819/8819 [==============================] - 2s 253us/step - loss: 0.1628 - acc: 0.9357 - val_loss: 0.1979 - val_acc: 0.9250
Epoch 7/10
8819/8819 [==============================] - 2s 257us/step - loss: 0.1545 - acc: 0.9400 - val_loss: 0.1979 - val_acc: 0.9264
Epoch 8/10
8819/8819 [==============================] - 2s 

## Evaluation - Validation Set

In [13]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# # pre-processing test data
# from tqdm import tqdm
# preprocessed_synopsis = []
# # tqdm is for printing the status bar
# for sentance in df_valid['Comment'].values:
#     sentance = re.sub(r"http\S+", "", sentance)
#     sentance = BeautifulSoup(sentance, 'lxml').get_text()
#     sentance = decontracted(sentance)
#     sentance = re.sub("\S*\d\S*", "", sentance).strip()
#     sentance = re.sub('[^A-Za-z]+', ' ', sentance)
#     # https://gist.github.com/sebleier/554280
#     sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
#     preprocessed_synopsis.append(sentance.strip())
    
    
# df_valid['preprocessed_comments']=preprocessed_synopsis

# # creating X and Y
X_valid = df_valid[['Comment']]
y_valid = df_valid.drop(columns=['Comment'])

In [14]:
y_valid = np.array(y_valid)

# creating embedding for validation
sentence_embeddings_valid = bert_encoder.encode(X_valid_Q1['Comment'])
embedding_matrix_valid = np.asarray(sentence_embeddings_valid)

In [15]:
embedding_matrix_valid.shape

(2594, 1024)

In [16]:
# saving these embeddings to file
np.save('../models/bert_x_valid_embeddings', embedding_matrix_valid)

In [25]:
score = model.evaluate(embedding_matrix_valid,y_valid)
score

2594/2594 [==============================] - 3s 1ms/step


[0.195380519837164, 0.9268183219221434]

## Evaluation - Precision & Recall

In [26]:
pred = model.predict(embedding_matrix_valid, batch_size=batch_size, verbose=1)

2594/2594 [==============================] - 1s 215us/step


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

For threshold:  0.1
Micro-average quality numbers
Precision: 0.4377, Recall: 0.8389, F1-measure: 0.5752


For threshold:  0.2
Micro-average quality numbers
Precision: 0.5554, Recall: 0.7645, F1-measure: 0.6434


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6301, Recall: 0.6979, F1-measure: 0.6623


For threshold:  0.4
Micro-average quality numbers
Precision: 0.6918, Recall: 0.6434, F1-measure: 0.6668


For threshold:  0.5
Micro-average quality numbers
Precision: 0.7330, Recall: 0.5826, F1-measure: 0.6492


For threshold:  0.6
Micro-average quality numbers
Precision: 0.7842, Recall: 0.5334, F1-measure: 0.6350


For threshold:  0.7
Micro-average quality numbers
Precision: 0.8311, Recall: 0.4760, F1-measure: 0.6053


For threshold:  0.8
Micro-average quality numbers
Precision: 0.8675, Recall: 0.4107, F1-measure: 0.5575


For threshold:  0.9
Micro-average quality numbers
Precision: 0.9102, Recall: 0.3195, F1-measure: 0.4730




### Testing RNN

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[512])
    layer = Embedding(max_features,50,input_length=512)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(12,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(embedding_matrix, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.05)

### New RNN

In [32]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 150
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 1
embed_size = 1024 # for BERT Large
n_class = 12

In [36]:
model = Sequential()
model.add(Embedding(max_features, embed_size,input_length = embed_size))
model.add(GRU(256, dropout=0.9, return_sequences=True))
model.add(GRU(256, dropout=0.9))
model.add(Dense(n_class, activation='sigmoid'))

In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1024, 1024)        10625024  
_________________________________________________________________
gru_5 (GRU)                  (None, 1024, 256)         983808    
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               393984    
_________________________________________________________________
dense_5 (Dense)              (None, 12)                3084      
Total params: 12,005,900
Trainable params: 12,005,900
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(embedding_matrix, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.15)

Train on 8819 samples, validate on 1557 samples
Epoch 1/1


INFO:plaidml:Analyzing Ops: 462 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 462 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 1128 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 1128 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 1782 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 1782 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 2522 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 2522 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 3326 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 3326 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 4273 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 4273 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 8604 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 8604 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 12509 of 171283 operations complete
INFO:plaidml:Analyzing Ops: 12509 of 171283 operations c

KeyboardInterrupt: 

## New Model Testing ends

**Score on Validation Data**

In [ ]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y

X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)

# creating padded dataset for x_valid
encoded_docs_valid = vect.texts_to_sequences(X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')
print(padded_docs_valid)

In [ ]:
score = model.evaluate(padded_docs_valid,y_valid)

In [ ]:
score

## Saving Model

In [ ]:
model.save('../models/cnn_use_sub_themes.pkl')

## Precision & Recall

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)

In [ ]:
#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))